# Notebook de Paul

Ici, Paul peut faire ses tests pour les parties du sujet dont il s'occupe.

Bon courage Paul :)

## Import necessary libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn

## Load data

In [115]:
%%bash
mkdir -p tmp_data
rm -rf tmp_data/*
mkdir -p tmp_data/train_set
mkdir -p tmp_data/test_set

In [116]:
def separate_data(path, nb_elements=1):
    """
    Separate given data in `path` in two sets with `nb_elements` per class for test

    Parameters
    ----------
    path: str
        Path to directory where dataset files are stored.

    nb_elements: integer
        Defines the number of test images taken in each class of the dataset
    """
    import os
    import shutil
    import random

    labels_path = os.path.join(path, "labels.txt")
    images_path = path
    test_path = 'tmp_data/test_set'
    train_path = 'tmp_data/train_set'

    labels = np.empty((0, 1))
    names = np.empty((0, 1))
    with open(labels_path) as f:
        for current_label in f:
            if current_label == "\n":
                continue
            label, name = current_label.split(": ")
            labels = np.append(labels, label)
            names = np.append(names, name)

    for label in labels:
        curr_test_path = os.path.join(test_path, label)
        curr_train_path = os.path.join(train_path, label)
        os.makedirs(curr_test_path, exist_ok=True)
        os.makedirs(curr_train_path, exist_ok=True)

        current_path = os.path.join(images_path, label)
        datas = os.listdir(current_path)
        random.shuffle(datas)

        count = 0
        for data in datas:
            data_path = os.path.join(current_path, data)
            if count < nb_elements:
                shutil.copyfile(data_path, os.path.join(curr_test_path, data))
                count += 1
            else:
                shutil.copyfile(data_path, os.path.join(curr_train_path, data))

    return labels, names

In [119]:
labels, names = separate_data('ressources/train', 2)
labels, names

(array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
        '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
        '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
        '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
        '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55',
        '56', '57'], dtype='<U32'),
 array(['lunettes\n', 'cible\n', 'sens interdit\n', 'clown\n', 'ancre\n',
        'bonhomme de neige\n', 'Ã©clair\n', 'araignÃ©e\n',
        "point d'exclamation\n", "point d'interrogation\n", 'tÃ¢ches\n',
        'fantÃ´me\n', 'bombe\n', 'carotte\n', 'dragon\n', 'arbre\n',
        'Å“il\n', 'voiture\n', 'lune\n', 'petit bonhomme\n', 'horloge\n',
        'cÅ“ur\n', "toÃ®le d'araignÃ©e\n", 'oiseau\n', 'feu\n',
        'ying yang\n', 'cactus\n', 'goutte\n', 'cadenas\n', 'marteau\n',
        'pomme\n', 'crayon\n', 'tÃªte de mort\n', 'chien\n', 'flocon\n',
        'fleur\n', 'dauphin\n', 'ma

In [120]:
def get_datasets(kind='train'):
    """
    Returns `kind` set and their label

    Parameters
    ----------
    kind: str (train or test)
        Type of dataset we want to get
    """
    data_path = 'tmp_data/train_set' if kind == 'train' else 'tmp_data/test_set'
    

In [123]:
get_datasets('train')

tmp_data/train_set
